In [1]:
import pandas as pd
import joblib
import numpy as np
import pickle

In [2]:
data = pd.read_csv("Dataset_ThuDuc_9202182025.csv")

# I. Chuẩn bị dữ liệu gồm 5000 dòng cuối cùng

In [3]:
num_col = ["temp", "wind", "RH", "P", "co", "no", "no2", "o3", "so2", "nh3"]
data_demo = data[-5000:].copy()
data_demo[num_col] = data_demo[num_col].clip(lower=0)

In [4]:
# Cái này tạo dữ liệu fake nha, tại tui lỡ lưu trong file pipeline có mấy dòng này
target_cols = ['pm2_5_next1', 'pm10_next1', 'pm2_5_next2', 'pm10_next2', 'pm2_5_next3', 'pm10_next3']
data_demo[target_cols] = np.nan

# II. Xử lí dữ liệu

In [5]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from statsmodels.tsa.seasonal import STL
from scipy.interpolate import interp1d
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

cols_in_ordered = ['time', 'temp', 'weather', 'wind', 'RH', 'P', 'co', 'no', 'no2', 'o3', 'so2', 'nh3', 'pm2_5_next1', 
                   'pm10_next1', 'pm2_5_next2', 'pm10_next2', 'pm2_5_next3', 'pm10_next3']

class FeaturesInOrder(BaseEstimator, TransformerMixin):
    def __init__(self, cols_in_ordered):
        self.cols_in_ordered = cols_in_ordered

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X = X[self.cols_in_ordered]
        return X

            
class MissingValueFiller(BaseEstimator, TransformerMixin):
    def __init__(self, cols_in_ordered = cols_in_ordered, num_cols=None, cat_cols=None, time_col="time", cat_window=4):
        self.cols_in_ordered = cols_in_ordered
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.time_col = time_col
        self.cat_window = cat_window
        self.most_common_per_hour_ = {} 

    def fit(self, X, y=None):
        X = X.copy()
        if self.cat_cols:
            X['hour'] = X[self.time_col].dt.hour
            
            for col in self.cat_cols:
                self.most_common_per_hour_[col] = X.groupby('hour')[col].agg(
                    lambda x: x.mode().tolist() if not x.mode().empty else [np.nan]
                ).to_dict()

        return self

    def transform(self, X):
        X = X.copy()
        
        # --- Numerical ---
        if self.num_cols:
            for col in self.num_cols:
                X[col] = X[col].mask(X[col] < 0, np.nan)
                X[col] = X[col].interpolate(method='cubic', limit_direction='both')
                X[col] = X[col].ffill().bfill()
        
        # --- Categorical ---
        if self.cat_cols:
            X[self.time_col] = pd.to_datetime(X[self.time_col], errors='coerce')
            X['hour'] = X[self.time_col].dt.hour
            for col in self.cat_cols:
                vals = X[col].tolist()
                col_filled = []
                for i in range(len(vals)):
                    start = max(0, i - self.cat_window//2)
                    end = min(len(vals), i + self.cat_window//2 + 1)
                    window_vals = [v for v in vals[start:end] if pd.notna(v)]
                    if window_vals:
                        col_filled.append(pd.Series(window_vals).mode()[0])
                    else:
                        col_filled.append(np.nan)
                X[col] = col_filled

                # Điền NaN còn lại theo mode cùng giờ
                X[col] = X.apply(
                    lambda row: self.most_common_per_hour_[col].get(row['hour'], [np.nan])[0]
                    if pd.isna(row[col]) else row[col], axis=1
                )
            X = X.drop(columns=['hour'])
        
        return X

class TSOutlierRemover(BaseEstimator, TransformerMixin):
    """
    Detect and replace outliers in numeric columns of a DataFrame using
    STL decomposition + Tukey IQR method.
    Q1 and Q3 are stored during fit() and reused in transform() for val/test.
    """
    def __init__(self, num_cols=None, seasonal_period=1000, seasonal_strength_threshold=0.6, k=3):
        """
        num_cols: list of numeric columns to process
        seasonal_period: period for STL decomposition
        seasonal_strength_threshold: threshold to decide if seasonality adjustment is used
        k: multiplier for IQR to detect outliers (Q1 - k*IQR, Q3 + k*IQR)
        """
        self.num_cols = num_cols
        self.seasonal_period = seasonal_period
        self.seasonal_strength_threshold = seasonal_strength_threshold
        self.k = k
        self.q1_ = {}
        self.q3_ = {}

    def fit(self, X, y=None):
        if self.num_cols is None:
            self.num_cols = X.select_dtypes(include=np.number).columns.tolist()

        for col in self.num_cols:
            x = X[col].values
            n = len(x)

            # STL decomposition
            if n >= 2 * self.seasonal_period:
                stl = STL(x, period=self.seasonal_period, robust=True)
                res = stl.fit()
                seasonal = res.seasonal
                trend = res.trend
                var_no_season = np.var(x - trend)
                var_with_season = np.var(x - trend - seasonal)
                F_s = 1 - var_with_season / var_no_season
                x_adj = x - seasonal if F_s > self.seasonal_strength_threshold else x
            else:
                x_adj = x

            # Trend estimation
            frac = min(0.3, 20 / n)
            trend_est = lowess(x_adj, np.arange(n), frac=frac, return_sorted=False)

            # Remainder
            remainder_est = x_adj - trend_est

            # Tukey IQR
            Q1 = np.percentile(remainder_est, 25)
            Q3 = np.percentile(remainder_est, 75)
            self.q1_[col] = Q1
            self.q3_[col] = Q3

        return self

    def transform(self, X, y=None):
        X_clean = X.copy()

        for col in self.num_cols:
            x = X_clean[col].values
            n = len(x)

            # STL decomposition
            if n >= 2 * self.seasonal_period:
                stl = STL(x, period=self.seasonal_period, robust=True)
                res = stl.fit()
                seasonal = res.seasonal
                trend = res.trend
                var_no_season = np.var(x - trend)
                var_with_season = np.var(x - trend - seasonal)
                F_s = 1 - var_with_season / var_no_season
                x_adj = x - seasonal if F_s > self.seasonal_strength_threshold else x
            else:
                x_adj = x

            # Trend estimation
            frac = min(0.3, 20 / n)
            trend_est = lowess(x_adj, np.arange(n), frac=frac, return_sorted=False)

            # Remainder
            remainder_est = x_adj - trend_est

            # Use stored Q1, Q3 from fit
            Q1 = self.q1_[col]
            Q3 = self.q3_[col]
            IQR = Q3 - Q1
            lower_bound = Q1 - self.k * IQR
            upper_bound = Q3 + self.k * IQR

            outliers = (remainder_est < lower_bound) | (remainder_est > upper_bound)

            # Replace outliers by linear interpolation
            x_clean_col = x.copy()
            if np.any(outliers):
                idx = np.arange(n)
                f = interp1d(idx[~outliers], x_clean_col[~outliers], kind='linear', fill_value="extrapolate")
                x_clean_col[outliers] = f(idx[outliers])

            X_clean[col] = x_clean_col

        return X_clean
    
class ClearNegativeValue(BaseEstimator, TransformerMixin):
    def __init__(self, positive_col):
        self.positive_col = positive_col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()  

        for col in self.positive_col:
            X.loc[X[col] < 0, col] = 0

        return X
    
class FixSkewedColumn(BaseEstimator, TransformerMixin):
    def __init__(self, skewed_col):
        self.skewed_col = skewed_col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()  

        X[self.skewed_col] = X[self.skewed_col].apply(lambda x: np.log1p(x))
        return X
    
class RankEncodeFeature(BaseEstimator, TransformerMixin):
    def __init__(self, cat_col, target_col, time_col=None):
        self.cat_col = cat_col
        self.target_col = target_col
        self.time_col = time_col
        self.rank_maps = {}

    def fit(self, X, y=None):
        X = X.copy()

        for target in self.target_col:
            for cat in self.cat_col:
                mean_target = X.groupby(cat)[target].mean()
                rank_map = mean_target.rank().to_dict()
                new_col = f"{cat}_{target}"
                self.rank_maps[new_col] = rank_map

        return self

    def transform(self, X, y=None):
        X = X.copy()

        for target in self.target_col:
            for cat in self.cat_col:
                new_col = f"{cat}_{target}"
                mapped = X[cat].map(self.rank_maps.get(new_col, {}))
                rank_values = list(self.rank_maps.get(new_col, {}).values())
                if len(rank_values) > 0:
                    mode_val = pd.Series(rank_values).mode()[0]
                else:
                    mode_val = 0

                X[new_col] = mapped.fillna(mode_val)

        X = X.drop(columns=self.cat_col, errors="ignore")


        return X
    
class ExtractTime(BaseEstimator, TransformerMixin):
    def __init__(self, time_col):
        self.time_col = time_col
    
    def fit(self, X, y=None):
        return self  
    
    def transform(self, X, y=None):
        df = X.copy()
        
        time_series = pd.to_datetime(df[self.time_col], errors='coerce')
        df['year'] = time_series.dt.year
        df['month'] = time_series.dt.month
        df['day'] = time_series.dt.day
        df['dayofweek'] = time_series.dt.dayofweek
        df['hour'] = time_series.dt.hour
        
        if self.time_col in df.columns: 
            df = df.drop(columns=[self.time_col])

        return df
    
class MinMaxScalerDF(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols
        self.scaler = MinMaxScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X[self.cols])
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X[self.cols] = self.scaler.transform(X[self.cols])
        return X

In [7]:
pre_pipeline = joblib.load("full_preprocess_pipeline.pkl")
fe_pipeline  = joblib.load("full_featureengineer_pipeline.pkl")

In [8]:
data_demo = pre_pipeline.transform(data_demo)
data_demo = fe_pipeline.transform(data_demo)
data_demo

/opt/anaconda3/lib/python3.13/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


,temp,wind,RH,P,co,no,no2,o3,so2,nh3,...,weather_pm10_next1,weather_pm2_5_next2,weather_pm10_next2,weather_pm2_5_next3,weather_pm10_next3,year,month,day,dayofweek,hour
30054,0.350000,0.173317,0.570954,0.578947,0.574044,0.086456,0.861759,0.462123,0.607105,0.755226,...,34.0,29.0,29.0,25.0,26.0,2025,2,4,1,22
30055,0.350000,0.099038,0.570954,0.578947,0.609408,0.287793,0.882092,0.196569,0.685807,0.776168,...,34.0,29.0,29.0,25.0,26.0,2025,2,4,1,23
30056,0.350000,0.173317,0.570954,0.578947,0.604275,0.376054,0.873869,0.053347,0.697405,0.795237,...,34.0,29.0,29.0,25.0,26.0,2025,2,5,2,0
30057,0.296846,0.189302,0.570108,0.552555,0.582648,0.387079,0.858163,0.031708,0.685807,0.812741,...,29.0,22.0,26.0,22.0,23.0,2025,2,5,2,1
30058,0.250000,0.173317,0.570954,0.526316,0.579812,0.433547,0.858163,0.010621,0.711330,0.839741,...,29.0,22.0,26.0,22.0,23.0,2025,2,5,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35049,0.350000,0.148557,0.886801,0.315789,-0.121345,0.001739,0.280912,0.549483,-0.192124,0.519720,...,34.0,29.0,29.0,25.0,26.0,2025,8,31,6,19
35050,0.400000,0.222836,0.813913,0.368421,-0.110892,0.001739,0.293103,0.539936,-0.184537,0.532943,...,34.0,29.0,29.0,25.0,26.0,2025,8,31,6,20
35051,0.400000,0.321874,0.813913,0.368421,-0.099145,0.001739,0.303729,0.531447,-0.178616,0.546221,...,29.0,22.0,26.0,22.0,23.0,2025,8,31,6,21
35052,0.350000,0.272355,0.886801,0.421053,-0.090165,0.001739,0.312170,0.526035,-0.174257,0.554318,...,29.0,22.0,26.0,22.0,23.0,2025,8,31,6,22


# III. Tạo đặc trưng mới

In [9]:
def add_lag_feature(data, lag_list, cols_added, time_col="time"):
    df = data.copy()
    cols_valid = [c for c in cols_added if c in df.columns]
    lag_dict = {}

    for col in cols_valid:
        for lag in lag_list:
            new_col = f"{col}_lag_{lag}"
            lag_dict[new_col] = df[col].shift(lag)

    df = pd.concat([df, pd.DataFrame(lag_dict)], axis=1)
    
    return df

def add_rolling_mean(data, rolling_list, cols_added, time_col="time"):
    df = data.copy()
    cols_valid = [c for c in cols_added if c in df.columns]
    roll_dict = {}

    for col in cols_valid:
        for roll in rolling_list:
            new_col = f"{col}_rollmean_{roll}"
            roll_dict[new_col] = df[col].rolling(window=roll).mean()

    df = pd.concat([df, pd.DataFrame(roll_dict)], axis=1)

    return df

In [10]:
lag_list = [1,3,6,12,24,36,48]
rolling_list = [r*720 for r in [1,3,6]] + [168,336,504]

data_demo = add_lag_feature(data_demo, lag_list, num_col+['weather0', 'weather1'])
data_demo = add_rolling_mean(data_demo, rolling_list, num_col)
data_demo

,temp,wind,RH,P,co,no,no2,o3,so2,nh3,...,so2_rollmean_4320,so2_rollmean_168,so2_rollmean_336,so2_rollmean_504,nh3_rollmean_720,nh3_rollmean_2160,nh3_rollmean_4320,nh3_rollmean_168,nh3_rollmean_336,nh3_rollmean_504
30054,0.350000,0.173317,0.570954,0.578947,0.574044,0.086456,0.861759,0.462123,0.607105,0.755226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30055,0.350000,0.099038,0.570954,0.578947,0.609408,0.287793,0.882092,0.196569,0.685807,0.776168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30056,0.350000,0.173317,0.570954,0.578947,0.604275,0.376054,0.873869,0.053347,0.697405,0.795237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30057,0.296846,0.189302,0.570108,0.552555,0.582648,0.387079,0.858163,0.031708,0.685807,0.812741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30058,0.250000,0.173317,0.570954,0.526316,0.579812,0.433547,0.858163,0.010621,0.711330,0.839741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35049,0.350000,0.148557,0.886801,0.315789,-0.121345,0.001739,0.280912,0.549483,-0.192124,0.519720,...,-0.002860,-0.189294,-0.160620,-0.168196,0.593562,0.403845,0.391879,0.524931,0.588046,0.566222
35050,0.400000,0.222836,0.813913,0.368421,-0.110892,0.001739,0.293103,0.539936,-0.184537,0.532943,...,-0.002954,-0.189320,-0.160747,-0.168202,0.593446,0.403899,0.391938,0.524720,0.587842,0.566235
35051,0.400000,0.321874,0.813913,0.368421,-0.099145,0.001739,0.303729,0.531447,-0.178616,0.546221,...,-0.003045,-0.189380,-0.160887,-0.168202,0.593329,0.403966,0.392001,0.524449,0.587618,0.566246
35052,0.350000,0.272355,0.886801,0.421053,-0.090165,0.001739,0.312170,0.526035,-0.174257,0.554318,...,-0.003142,-0.189439,-0.161021,-0.168196,0.593214,0.404045,0.392053,0.524172,0.587387,0.566248


In [11]:
all_target_col  = ["pm2_5_next1", "pm10_next1", "pm2_5_next2", "pm10_next2", "pm2_5_next3", "pm10_next3"]
X_test          = data_demo.drop(columns=all_target_col)
X_test          = X_test[-1:]
X_test

,temp,wind,RH,P,co,no,no2,o3,so2,nh3,...,so2_rollmean_4320,so2_rollmean_168,so2_rollmean_336,so2_rollmean_504,nh3_rollmean_720,nh3_rollmean_2160,nh3_rollmean_4320,nh3_rollmean_168,nh3_rollmean_336,nh3_rollmean_504
35053,0.35,0.272355,0.813913,0.421053,-0.08692,0.001739,0.316259,0.521504,-0.175702,0.557003,...,-0.003254,-0.189465,-0.161134,-0.168179,0.593109,0.404132,0.392086,0.523938,0.587159,0.566251


# IV. Predict pm2.5

In [12]:
with open("best_pm2.5_xgb.pkl", "rb") as f:
    model_pm2_5 = pickle.load(f)

y_pred = model_pm2_5.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=['pm2_5_next1', 'pm2_5_next2', 'pm2_5_next3'])
y_pred

/var/folders/ds/lr5ykbnn4sxcmmgzlsgh4z3r0000gn/T/ipykernel_94514/4079929426.py:2: UserWarning: [12:12:43] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  model_pm2_5 = pickle.load(f)


,pm2_5_next1,pm2_5_next2,pm2_5_next3
0,19.595964,20.637787,24.134983


# V. Predict pm10

In [13]:
with open("best_pm10_rf.pkl", "rb") as f:
    model_pm10 = pickle.load(f)

y_pred = model_pm10.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=['pm10_next1', 'pm10_next2', 'pm10_next3'])
y_pred

,pm10_next1,pm10_next2,pm10_next3
0,19.456901,24.208528,27.83673
